# How to georeference a BIM model

IFC files are an open standard for exchanging Building Information Models (BIM) that contain geometric and non-geometric information about constructions. 
This notebook shows how IFC files can be georeferenced by providing **7 parameters** that are part of the IFC standard but are often omitted. 
The use of these parameters will make it a lot easier to **combine BIM and geospatial data** (e.g. cadastral parcel data, 3D city models, point cloud data, or a digital elevation model). 
This enables many use cases in areas such as construction planning, environmental permits, or asset management. 

## The problem
One of the impediments to the automated processing of IFC data is the fact that georeferencing information is not included in IFC information exchanges. Two scenarios are possible:
 1. **Local reference system**: in most construction projects, the partners share a common *local* reference system, which is defined as a local origin and axis orientation. The local origin is often a conspicuous point near the construction site and the axis orientation is often chosen for convenience (e.g. along the axis of the main construction). The geolocation of the local reference system is known to the partners - and used to combine BIM models - but not explicitly included in the IFC file.
 1. **Geospatial reference system**: In other construction projects, the partners exchange IFC files in a standard projected reference system used by land surveyors or geospatial data (e.g. [EPSG:2169](https://epsg.io/2169) Luxembourg).  Although this information is known to the partners in the project, it is often left implicit in the IFC file thus inhibiting the possibility for automated processing.
 
## The solution
In both scenarios, the georeferencing information can be made explicit by using the **IfcProjectedCRS** and **IfcMapConversion** entities from the [IFC4 standard](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/templates/project-global-positioning.htm). The former allows specifiying a recognized, target coordinate reference system (CRS). The latter can be used to define the coordinate transformation needed to convert coordinates from the project CRS to the specified target CRS. As depicted in the diagram below from the IFC standard, these ifc entities are directly related to the IfcProject (a subclass of IfcContext) entity.

[![BuildingSMART IFC4](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/templates/diagrams/project-global-positioning.png)](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/templates/project-global-positioning.htm)

[IfcProjectedCRS](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/ifcrepresentationresource/lexical/ifcprojectedcrs.htm) allows specifying a target projected coordinate reference system. The most relevant attribute is the name of the reference system:
* **IfcProjectedCRS.Name**: According to the IFC4 specification, the name shall be taken from the list recognized by the European Petroleum Survey Group EPSG and should be qualified by the EPSG name space, for example as '[EPSG:2169](https://epsg.io/2169)'.

[IfcMapConversion](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/ifcrepresentationresource/lexical/ifcmapconversion.htm) provides information on how the reference system of the IFC file can be converted into the reference system indicated by IfcProjectedCRS. It contains information on the required coordinate translation, rotation, and scaling:
* **IfcMapConversion.Eastings, IfcMapConversion.Northings, IfcMapConversion.OrthogonalHeight** indicate the position of the origin of the local reference system used in the IFC file in terms of the indicated IfcProjectedCRS. Together, they define the coordinate *translation* needed to convert the *origin* of the local project CRS into the origin of the target geospatial CRS.
* **IfcMapConversion. XAxisAbscissa and IfcMapConversion.XAxisOrdinate**: together define the angle of *rotation* required to convert the axis orientation of the local reference system, into the axis orientation of the indicated coordinate reference system.
* **IfcMapConversion.Scale**: Scale to be used, when the units of the CRS are not identical to the units of the engineering coordinate system (often expressed in metres). If omitted, the value of 1.0 is assumed.

In IFC2x3 the same information can be included by using equivalent property sets on IfcSite as proposed by BuildingSMART Australasia in its paper '[User guide for georeferencing in IFC](https://www.buildingsmart.org/wp-content/uploads/2020/02/User-Guide-for-Geo-referencing-in-IFC-v2.0.pdf)'. 


## Explore georeference information with IfcOpenShell-python

Let's use [IfcOpenShell](https://github.com/IfcOpenShell/IfcOpenShell) to show how to read this georeference information from an IFC file that is already properly georeferenced.
For this purpose, we have taken the [Duplex Apartment](https://github.com/buildingSMART/Sample-Test-Files/tree/master/IFC%202x3/Duplex%20Apartment) IFC file by BuildingSMART International and have enhanced it with proper - yet entirely ficticious - georeferencing information. 

### Visualize the building

First, let's visualize the rooms (IfcSpace) in the IFC file. The resulting threeJS viewer shows the XYZ axis. Its origin is clearly located very near the construction site. 



In [10]:
from math import atan2, degrees
import ifcopenshell
import ifcopenshell.geom
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer, format_color

settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)
settings.set(settings.USE_BREP_DATA, True)
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

# Open the IFC file using IfcOpenShell
fn = './Duplex_A_20110907_georeferenced.ifc'
ifc_file = ifcopenshell.open(fn)

threejs_renderer = JupyterRenderer(size=(500, 500))
spaces = ifc_file.by_type("IfcSpace")

for space in spaces:
    if space.Representation is not None:
        shape = ifcopenshell.geom.create_shape(settings, inst=space)
        r,g,b,alpha = shape.styles[0]
        color = format_color(int(abs(r)*255), int(abs(g)*255), int(abs(b)*255))
        threejs_renderer.DisplayShape(shape.geometry, shape_color = color, transparency=True, opacity=alpha, render_edges=True)

threejs_renderer.Display()

/opt/conda/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


### Extract georeference information

The below function `map_conversion_crs()` can be used to extract georeferencing information from an IFC file. From XAxisAbscissa and XAxisOrdinate we calculate the rotation.

In [11]:
import ifcopenshell.util.pset
import ifcopenshell.util.geolocation
from ifcopenshell.util.element import get_psets
import pandas as pd
from IPython.display import display

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
        
def map_conversion_crs(ifc_file):
    map_conversion = None
    crs = None
    if ifc_file.schema == 'IFC4':
        project = ifc_file.by_type("IfcProject")[0]
        for c in (m for c in project.RepresentationContexts for m in c.HasCoordinateOperation):
            return c, c.TargetCRS
    if ifc_file.schema == 'IFC2X3':
        site = ifc_file.by_type("IfcSite")[0]
        psets = get_psets(site)
        if 'ePSet_MapConversion' in psets.keys() and 'ePSet_ProjectedCRS' in psets.keys():
            return Struct(**psets['ePSet_MapConversion']), Struct(**psets['ePSet_ProjectedCRS'])
    return map_conversion, crs
        
IfcMapConversion, IfcProjectedCRS = map_conversion_crs(ifc_file)

df = pd.DataFrame(list(IfcProjectedCRS.__dict__.items()) + list(IfcMapConversion.__dict__.items()), columns= ['property', 'value'])
display(df)

rotation = degrees(atan2(IfcMapConversion.XAxisOrdinate, IfcMapConversion.XAxisAbscissa))
print(f'Rotation is: {rotation}°  (degrees(atan2(map_conversion.XAxisOrdinate, map_conversion.XAxisAbscissa)) ')



,property,value
0,Name,EPSG:2169
1,Eastings,76670.0
2,Northings,77179.0
3,OrthogonalHeight,293.700012
4,XAxisAbscissa,0.945519
5,XAxisOrdinate,-0.325568
6,Scale,1.0


Rotation is: -19.000001223867017°  (degrees(atan2(map_conversion.XAxisOrdinate, map_conversion.XAxisAbscissa)) 


### Convert 3D spaces into 2D polygons (footprint)

We will use a utility function `shape_to_polygons(shape)` that will convert a 3D space into a 2D polygon, by extracting the shape into faces. Only the first face that is converted into a valid polygon is taken. We then use [GeoPandas](https://geopandas.org) to convert the polygons from the project CRS into the geospatial CRS using the IfcMapConversion parameters. We will respectively apply a rotation, translation, and scaling. The resulting data is pure geospatial data that can be depicted on a map and combined with other geospatial data, in this example with OpenStreetMap data.

In [16]:
from util import shape_to_polygons

polygons = []
names = []
for space in spaces:
    if space.Representation is not None:
        shape = ifcopenshell.geom.create_shape(settings, inst=space)
        shape_polygons = shape_to_polygons(shape)
        polygons = polygons + shape_polygons
        names = names + [space.Name for _ in shape_polygons]
        
import geopandas as gpd
from geopandas import GeoSeries
import folium

footprint = GeoSeries(polygons,crs=IfcProjectedCRS.Name)
footprint = footprint\
    .rotate(rotation + 90, origin=(0,0,), use_radians=False)\
    .translate(IfcMapConversion.Eastings, IfcMapConversion.Northings, 0)\
    .scale(IfcMapConversion.Scale)

df = gpd.GeoDataFrame({'name': names}, geometry=footprint)
df = df.to_crs(epsg=4326)


m = folium.Map(location=[49.629, 6.122], zoom_start=19, tiles='CartoDB positron', max_zoom=30)
for _, r in df.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['name']).add_to(geo_j)
    geo_j.add_to(m)
m

## Conclusion

The use of  **IfcProjectedCRS** and **IfcMapConversion** should be included in BIM information exchange agreements.


## References

BuildingSMART. IFC Specifications database. https://technical.buildingsmart.org/standards/ifc/ifc-schema-specifications/ 

BuildingSMART Australasia (2020). User Guide for Geo-referencing in IFC, version 2.0. https://www.buildingsmart.org/wp-content/uploads/2020/02/User-Guide-for-Geo-referencing-in-IFC-v2.0.pdf

Dion Moult (2019). IFC Coordinate Reference Systems and Revit. https://thinkmoult.com/ifc-coordinate-reference-systems-and-revit.html 

Clemen, C., & Görne, H. (2019). Level of Georeferencing (LoGeoRef) using IFC for BIM. Journal of Geodesy, Cartography and Cadastre, (10). https://jgcc.geoprevi.ro/docs/2019/10/jgcc_2019_no10_3.pdf

Noardo, F., Krijnen, T., Arroyo Ohori, K., Biljecki, F., Ellul, C., Harrie, L., Eriksson, H., Polia, L., Salheb, N., Tauscher, H., van Liempt, J., Goerne, H., Hintz, D., Kaiser, T., Leoni, C., Warchol, A., Stoter, J. (2021). Reference study of IFC software support: the GeoBIM benchmark 2019 – Part I. Transactions in GIS.